# Tensorflow2 Object Detection API 사용하여 객체 탐지 모델 만들기
지금부터 Tensorflow2 Object Detection API를 사용하여 사용자 지정 데이터에 대한 객체 탐지 모델을 만들도록 하겠습니다.

작업을 시작하기 위해서 학습할 이미지 사진과 라벨링 정보가 들어있는 xml 파일이 필요합니다.

우선 부표와 작은공을 학습시킬 예정이지만 주석으로 표시한 부분을 수정하면 다른 데이터로 학습시킬 수 있습니다.

작업을 시작하기에 앞서 빠른 환경에서 작업하기 위해 GPU를 사용하도록 설정합니다.

런타임 > 런타임 유형 변경 > 하드웨어 가속기 > GPU 

설정이 끝났으면 본격적으로 시작합니다.

( 참고 : https://ichi.pro/ko/tensorflow-gaegche-gamji-gaideu-tensorflow-2-252181752953859 )

( 전반적인 내용은 위 링크를 참고하여 작성하였습니다. )


---



구글 드라이브와 연결하는 코드입니다.

colab에서 생성한 파일들을 저장하고 최종적으로 생성된 모델 역시 이곳에 저장할 예정입니다.

colab은 일정시간 이후 런타임이 끊어지게 되는데 런타임이 끊어질 경우 드라이브에 업로드 하지 않은 파일들은 전부 사라지게 됩니다.

그러므로 한번에 모든 작업을 실행하기보단 작업 중간중간 생성된 파일들을 드라이브에 저장해가면서 작업하는 것을 추천합니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

각종 파일을 다운로드할 워크스페이스를 생성합니다.

드라이브에도 생성된 파일들을 저장할 폴더를 미리 생성합니다.

In [ ]:
%mkdir workspace
%mkdir /content/drive/MyDrive/shipdata_tf/data /content/drive/MyDrive/shipdata_tf/my_model
%cd /content/workspace

Tensorflow2 Object Detection API를 포함하고 있는 모델을 다운로드합니다.

Tensorflow2 Object Detection API는 객체 탐지 모델을 조금 더 간편하게 학습할 수 있도록 도와주는 API입니다.

해당 모델은 객체 탐지 외에도 다양한 학습을 지원하고 있습니다.

In [ ]:
!git clone --q https://github.com/tensorflow/models.git

추가적으로 필요한 라이브러리를 설치하고 모델을 사용할 준비를 합니다.

In [ ]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow lxml matplotlib
!pip install -qq pycocotools
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.

모델이 제대로 준비되었는지 테스트합니다.

마지막에 OK가 출력되면 정상적으로 준비된 것입니다.

(시간이 오래 걸립니다.)

In [ ]:
%cp object_detection/packages/tf2/setup.py .
!python -m pip install .
!python object_detection/builders/model_builder_tf2_test.py

라벨링 파일들을 구글 드라이브에 업로드 합니다.

In [ ]:
# 업로드한 사진, xml 파일 위치
drive_img = '/content/drive/MyDrive/shipdata_tf/img'
drive_xml = '/content/drive/MyDrive/shipdata_tf/xml'

# 생성할 csv, record 파일 위치
drive_csv = '/content/drive/MyDrive/shipdata_tf/data/labels.csv'
drive_tfrecord = '/content/drive/MyDrive/shipdata_tf/data/train.tfrecord'

업로드한 xml 파일들을 하나의 csv 파일로 통합합니다.

( 참고 : https://github.com/TannerGilbert/Tensorflow-Object-Detection-API-Train-Model/blob/master/xml_to_csv.py )

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main(drive_xml, drive_csv):
	xml_df = xml_to_csv(drive_xml)
	xml_df.to_csv(drive_csv, index=None)


main(drive_xml, drive_csv)

라벨링 파일을 Tensorflow에서 학습시킬 수 있는 tfrecord 파일로 만들어주는 코드를 실행합니다.

코드 중간에 클래스 이름을 본인에게 맞는 클래스 이름으로 수정해야 합니다.

( 참고 : https://github.com/TannerGilbert/Tensorflow-Object-Detection-API-Train-Model/blob/master/generate_tfrecord.py )

In [ ]:
#based on https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import sys
import pandas as pd

from tensorflow.python.framework.versions import VERSION
if VERSION >= "2.0.0a0":
    import tensorflow.compat.v1 as tf
else:
    import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict


CSV = drive_csv
RECORD = drive_record
IMG = drive_img


# 클래스 이름 수정
def class_text_to_int(row_label):
    if row_label == 'buoy':
        return 1
    elif row_label == 'smallball':
        return 2
    else:
        return None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main():
    writer = tf.python_io.TFRecordWriter(RECORD)
    path = os.path.join(IMG)
    examples = pd.read_csv(CSV)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    print('Successfully created the TFRecords: {}'.format(RECORD))


if __name__ == '__main__':
    main()

학습에 필요한 파일들을 저장할 폴더를 생성합니다.

In [ ]:
%cd /content/workspace/
%mkdir annotations exported-models pre-trained-models models/my_data

미리 학습된 모델을 다운로드 합니다.

이 과정은 객체 탐지 모델의 정확도와 인식 속도를 향상시켜줍니다.

In [ ]:
import tarfile
import os

%cd pre-trained-models
!curl "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz" --output "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"

model_file = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
tar = tarfile.open(model_file)
tar.extractall()
tar.close()
os.remove(model_file)

클래스 이름에 관한 내용이 저장된 pbtxt 파일을 생성하고 드라이브에 업로드 합니다.

pbtxt 파일의 형식은 아래를 참고합니다.

In [ ]:
'''
# labels.pbtxt
item {
  id: 1
  name: 'buoy'
}
item {
  id: 2
  name: 'smallball'
}
'''

학습에 사용할 tfrecord 파일과 pbtxt 파일을 작업공간에 복사합니다.

In [ ]:
# 업로드한 pbtxt 파일 위치
drive_pbtxt = '/content/drive/MyDrive/shipdata_tf/data/labels.pbtxt'

%cp $drive_record /content/workspace/annotations/train.tfrecord
%cp $drive_pbtxt /content/workspace/annotations/labels.pbtxt

학습과 관련된 내용은 pipeline.config 파일에 저장되어 있습니다.

이 파일을 살펴보도록 하겠습니다.

In [ ]:
%cat ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config

colab에서는 해당 파일을 더블클릭하여 수정할 수 있습니다.

수정할 내용은 아래를 참고합니다.

수정이 끝난 후 Ctrl + s 로 저장할 수 있습니다.

In [ ]:
'''
# pipeline.config
# 학습시킬 클래스의 개수
num_classes: 2

# 한번에 학습시킬 묶음
# 너무 클 경우 과부화가 발생합니다.
batch_size: 8

# 중간 저장 위치
fine_tune_checkpoint: "/content/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0"

# 총 학습시킬 횟수
# 25000번 정도 학습하는 것을 권장합니다.
num_steps: 25000

# 중간 저장
fine_tune_checkpoint_type: "detection"

# "PATH_TO_BE_CONFIGURED" 부분을 본인의 경로에 맞게 수정합니다.
# 각각 2줄씩 총 4줄을 수정합니다.
# 작업공간에 복사한 pbtxt 파일 위치
label_map_path: "/content/workspace/annotations/labels.pbtxt"

# 작업공간에 복사한 record 파일 위치
input_path: "/content/workspace/annotations/train.tfrecord"
'''

수정이 끝난 후 잘 저장되었나 확인합니다.

In [ ]:
%cat /content/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config

학습을 진행하기 위해 환경변수를 지정합니다.

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/workspace/models/:/content/workspace/models/research/:/content/workspace/models/research/slim/'

학습 진행 상황을 확인할 수 있는 텐서보드를 로딩합니다.

처음에는 빈화면이 나오지만 학습을 진행한 후 우측 상단의 새로고침을 누르면 학습 현황이 시각적으로 나타납니다.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/workspace/models/my_data'

학습 실행파일을 실행하기 편한 위치로 복사합니다.

In [ ]:
%cd /content/workspace
!cp '/content/workspace/models/research/object_detection/model_main_tf2.py' .

학습을 진행합니다.

30분 정도 걸렸습니다.

In [ ]:
!python model_main_tf2.py --model_dir=models/my_data --pipeline_config_path=/content/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config

가장 마지막 체크포인트만 저장합니다.

( 참고 : https://gist.github.com/rohitp934/8b1918bf3f13545bf667f86f2af77a6d#file-getlatestcheckpoint-py )

In [ ]:
import numpy as np
import re

output_directory = '/content/workspace/exported-models/'

lst = os.listdir("/content/workspace/models/my_data/")

lst = [l for l in lst if 'ckpt-' in l and '.index' not in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()]
last_model_path = os.path.join('/content/workspace/models/my_data', last_model)

마지막 체크포인트에서 데이터를 추출하여 모델을 생성합니다.

In [ ]:
!python /content/workspace/models/research/object_detection/exporter_main_v2.py \
--input_type=image_tensor \
--pipeline_config_path=/content/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config \
--output_directory=/content/drive/MyDrive/shipdata_tf/my_model \
--trained_checkpoint_dir=/content/workspace/models/my_data

모델이 잘 학습되었는지 확인합니다.

( 참고 : https://colab.research.google.com/github/TannerGilbert/Tensorflow-Object-Detection-API-Train-Model/blob/master/Tensorflow_2_Object_Detection_Train_model.ipynb#scrollTo=EEX-m3P1yp4y )

In [ ]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

def load_image_into_numpy_array(path):
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

category_index = label_map_util.create_category_index_from_labelmap(drive_pbtxt, use_display_name=True)
tf.keras.backend.clear_session()
# 모델 저장한 경로
model = tf.saved_model.load(f'/content/drive/MyDrive/shipdata_tf/my_model/saved_model')


def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]

  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  if 'detection_masks' in output_dict:
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict


# 테스트용 이미지 경로
for image_path in glob.glob('/content/drive/MyDrive/shipdata_tf/img/buoy1.jpg'):
  image_np = load_image_into_numpy_array(image_path)
  output_dict = run_inference_for_single_image(model, image_np)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)
  display(Image.fromarray(image_np))

학습시킨 Tensorflow 모델을 Jetson nano에서 사용하기 위해 더욱 가벼운 모델인 Tensorflow lite 모델로 변환합니다.

( 참고 : https://www.tensorflow.org/lite/convert/?hl=ko )

In [ ]:
import tensorflow as tf


# 모델 저장한 경로
saved_model_dir = '/content/drive/MyDrive/shipdata_tf/saved_model/saved_model'

model = tf.saved_model.load(saved_model_dir)
model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY].inputs[0].set_shape([1, 320, 320, 3])
tf.saved_model.save(model, "saved_model_updated", signatures=model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY])

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_updated', signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
print(input_shape)

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

만약 위 코드 실행중 오류 발생 시 모델 폴더(saved_model)를 다운로드한 후 밑의 내용을 커멘드 창에 입력합니다.

In [ ]:
'''
tflite_convert \
  # 모델 저장된 위치
  --saved_model_dir=/tmp/mobilenet_saved_model \
  # 모델 저장할 위치
  --output_file=/tmp/mobilenet.tflite
'''

여기까지 잘 실행했다면 성공적으로 .tflite 파일이 생성되었을 것입니다.

이를 다운받아서 사용하면 됩니다.